In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import requests
from bs4 import BeautifulSoup
import re
from gk_parser import *
import time
import pandas as pd

#get wikipedia page
url_list = ['2011','2012','2013','2014','2015','2016','2017','2018', '2019']
movie_list = []
for x in url_list:
    if x == '2019':
        generatedUrl = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2019'
    else:
        generatedUrl = 'https://en.wikipedia.org/wiki/' + x + '_in_film' 
    time.sleep(1)
    r = requests.get(generatedUrl)
    soup = BeautifulSoup(r.text, 'html.parser')

    #scrap data
    page = str(soup)
    #comment for myself [\w\(\)\%.\,\_\:\"]+ matches multiple occassion of those previous
    #<td> tag seems to exclude some (why?)
    search_tag = re.compile(r'<i><a href="/wiki/[\w\(\)\%\.\,\_\:\"]+ title="[\w\s\%\(\).\,\_\:\;\'\"\-\=]+"')
    result = search_tag.match(page)
    crazy = re.findall(search_tag, page)

    #clean the names from tags
    clean_left_tag = re.compile(r'<i><a href="/wiki/[\w\(\)\%\.\,\_\:\"]+ title="')
    clean_right_tag = re.compile(r'"')
    for x in range(len(crazy)):
        crazy[x] = re.sub(clean_left_tag,'',crazy[x])
        crazy[x] = re.sub(clean_right_tag,'',crazy[x])
    movie_list += crazy

#remove duplicates
movie_list = list(set(movie_list))

In [23]:
print(len(movie_list))

5771


In [24]:
#further filter the data
#the obvious one bad data is space+(film) 
badFilmPattern = re.compile('\s\(film\)')
#the other one is space+(YYYY film)
#this one is tricky and bad as with remakes which is usually associated with this labeling
#we can try both filmname_YYYY or just filmname
remakesPattern = re.compile(r'\(([0-9]+)\sfilm\)')
for x in range(len(movie_list)):
    movie_list[x] = re.sub(badFilmPattern,'', movie_list[x])
    searchResult = re.search(remakesPattern, movie_list[x])
    if searchResult:
        movie_list[x] = re.sub(remakesPattern, searchResult.group(1), movie_list[x])

#the main url format is lowercase all and replace space with underscores
spacePattern = re.compile('\s')
punctuationPattern = re.compile('[\,\:\.\']')

#remove all space and punctuations
for x in range(len(movie_list)):
    movie_list[x] = re.subn(spacePattern,'_', movie_list[x].lower())[0]
    movie_list[x] = re.subn(punctuationPattern,'', movie_list[x])[0]



In [49]:
#find equivalent rotten tomatoes site
rtSite = 'https://www.rottentomatoes.com/m/'

yearPattern = re.compile('_[0-9]{4}')
retryList = []
failCount = 0
progressCount = 0
totalSize = len(movie_list)
threshold = 0.1

compiledData = []
for x in range(len(movie_list)):
    time.sleep(0.2) #do not break my internet
    pageName = movie_list[x]
    newPage = rtSite + pageName
    r = requests.get(newPage)
    #check if 404 -- probably due to bad name
    if r.status_code == 404: #try without the year
        newPage = rtSite + re.sub(yearPattern,'',pageName)
        r = requests.get(newPage)
        if r.status_code == 404:
            retryList.append(newPage)
            print(newPage + ' is 404')
            failCount += 1
            progressCount += 1
            continue
    soup = BeautifulSoup(r.text, 'html.parser')
    page = str(soup)
    tempList = [pageName]
    tempList += parseDocument(page)
    compiledData.append(tempList)
    progressCount += 1
    if float(progressCount)/ float(totalSize) > threshold:
        print('Progress: ' + str(threshold))
        threshold += 0.1


https://www.rottentomatoes.com/m/the_hobbit_(film_series) is 404
https://www.rottentomatoes.com/m/the_naked_gun_2½_the_smell_of_fear is 404
https://www.rottentomatoes.com/m/the_blonde_from_peking is 404
https://www.rottentomatoes.com/m/a_matter_of_time is 404
https://www.rottentomatoes.com/m/collide is 404
https://www.rottentomatoes.com/m/the_secret_of_the_old_woods is 404
https://www.rottentomatoes.com/m/the_presidents_analyst is 404
https://www.rottentomatoes.com/m/200_motels is 404
https://www.rottentomatoes.com/m/the_blue_man is 404
https://www.rottentomatoes.com/m/80_steps_to_jonah is 404
https://www.rottentomatoes.com/m/the_sword_in_the_stone is 404
https://www.rottentomatoes.com/m/in_the_name_of_the_sovereign_people is 404
https://www.rottentomatoes.com/m/the_day_of_the_wolves is 404
https://www.rottentomatoes.com/m/the_revolt_of_mamie_stover is 404
https://www.rottentomatoes.com/m/hürriyet_daily_news is 404
https://www.rottentomatoes.com/m/my_friends is 404
https://www.rottento

https://www.rottentomatoes.com/m/nerdist_industries is 404
https://www.rottentomatoes.com/m/pacific_rim is 404
https://www.rottentomatoes.com/m/deep_cover is 404
https://www.rottentomatoes.com/m/the_st_valentines_day_massacre is 404
https://www.rottentomatoes.com/m/chéri is 404
https://www.rottentomatoes.com/m/the_tomorrow_man is 404
https://www.rottentomatoes.com/m/the_amorous_adventures_of_moll_flanders is 404
https://www.rottentomatoes.com/m/office_romance is 404
https://www.rottentomatoes.com/m/a_few_cubic_meters_of_love is 404
https://www.rottentomatoes.com/m/the_new_age is 404
https://www.rottentomatoes.com/m/the_human_centipede_2_(full_sequence) is 404
https://www.rottentomatoes.com/m/black_rain_(1989_american_film) is 404
https://www.rottentomatoes.com/m/list_of_james_bond_films is 404
https://www.rottentomatoes.com/m/the_getting_of_wisdom is 404
https://www.rottentomatoes.com/m/spotlight is 404
https://www.rottentomatoes.com/m/full_circle is 404
https://www.rottentomatoes.com/

https://www.rottentomatoes.com/m/super_fly_tnt is 404
https://www.rottentomatoes.com/m/20000_leagues_under_the_sea is 404
https://www.rottentomatoes.com/m/the_professor is 404
https://www.rottentomatoes.com/m/the_song_of_bernadette is 404
https://www.rottentomatoes.com/m/the_road_to_hong_kong is 404
https://www.rottentomatoes.com/m/the_reluctant_astronaut is 404
https://www.rottentomatoes.com/m/the_happiest_days_of_your_life is 404
https://www.rottentomatoes.com/m/the_fastest_gun_alive is 404
https://www.rottentomatoes.com/m/the_last_days_of_pompeii is 404
https://www.rottentomatoes.com/m/the_marshal_of_windy_hollow is 404
https://www.rottentomatoes.com/m/the_wild_angels is 404
https://www.rottentomatoes.com/m/nicholas_and_alexandra is 404
https://www.rottentomatoes.com/m/the_cruel_sea is 404
https://www.rottentomatoes.com/m/on_the_town is 404
https://www.rottentomatoes.com/m/sams_son is 404
https://www.rottentomatoes.com/m/a_bridge_too_far is 404
https://www.rottentomatoes.com/m/the_p

https://www.rottentomatoes.com/m/the_sundowners is 404
https://www.rottentomatoes.com/m/blood_feast is 404
https://www.rottentomatoes.com/m/beyond_the_mask is 404
https://www.rottentomatoes.com/m/the_color_purple is 404
https://www.rottentomatoes.com/m/the_odessa_file is 404
https://www.rottentomatoes.com/m/get_yourself_a_college_girl is 404
https://www.rottentomatoes.com/m/the_farmers_daughter is 404
https://www.rottentomatoes.com/m/the_ernie_game is 404
https://www.rottentomatoes.com/m/willies_war is 404
https://www.rottentomatoes.com/m/allmovie is 404
https://www.rottentomatoes.com/m/swerve is 404
https://www.rottentomatoes.com/m/harry_potter_(film_series) is 404
https://www.rottentomatoes.com/m/apaye is 404
https://www.rottentomatoes.com/m/the_preachers_wife is 404
https://www.rottentomatoes.com/m/mchales_navy is 404
https://www.rottentomatoes.com/m/the_milky_way is 404
https://www.rottentomatoes.com/m/honey_i_shrunk_the_kids_(franchise) is 404
https://www.rottentomatoes.com/m/the_

https://www.rottentomatoes.com/m/it_always_rains_on_sunday is 404
https://www.rottentomatoes.com/m/glory_road is 404
https://www.rottentomatoes.com/m/a_funny_thing_happened_on_the_way_to_the_forum is 404
https://www.rottentomatoes.com/m/furry_vengeance is 404
https://www.rottentomatoes.com/m/traffic is 404
https://www.rottentomatoes.com/m/screen_rant is 404
https://www.rottentomatoes.com/m/the_worlds_fastest_indian is 404
https://www.rottentomatoes.com/m/the_wonderful_world_of_the_brothers_grimm is 404
https://www.rottentomatoes.com/m/lies_my_father_told_me is 404
https://www.rottentomatoes.com/m/the_worlds_greatest_athlete is 404
https://www.rottentomatoes.com/m/escape_to_victory is 404
https://www.rottentomatoes.com/m/the_brides_of_dracula is 404
https://www.rottentomatoes.com/m/death_by_hanging is 404
https://www.rottentomatoes.com/m/les_anarchistes is 404
https://www.rottentomatoes.com/m/hawk_of_the_caribbean is 404
https://www.rottentomatoes.com/m/the_octagon is 404
https://www.ro

https://www.rottentomatoes.com/m/formula_of_love is 404
https://www.rottentomatoes.com/m/the_rock is 404
https://www.rottentomatoes.com/m/the_nutcracker_prince is 404
https://www.rottentomatoes.com/m/polis_evo is 404
https://www.rottentomatoes.com/m/anastasia is 404
https://www.rottentomatoes.com/m/the_people_vs_larry_flynt is 404
https://www.rottentomatoes.com/m/the_great_gatsby is 404
https://www.rottentomatoes.com/m/evil_in_the_time_of_heroes is 404
https://www.rottentomatoes.com/m/the_homecoming is 404
https://www.rottentomatoes.com/m/all_eyez_on_me is 404
https://www.rottentomatoes.com/m/prom is 404
https://www.rottentomatoes.com/m/silent_house is 404
https://www.rottentomatoes.com/m/many_wars_ago is 404
https://www.rottentomatoes.com/m/the_informers is 404
https://www.rottentomatoes.com/m/airplane_ii_the_sequel is 404
https://www.rottentomatoes.com/m/the_prestige is 404
https://www.rottentomatoes.com/m/the_trip_to_italy is 404
https://www.rottentomatoes.com/m/the_fallen_sparrow i

https://www.rottentomatoes.com/m/the_cable_guy is 404
https://www.rottentomatoes.com/m/the_family is 404
https://www.rottentomatoes.com/m/the_voice_of_the_moon is 404
https://www.rottentomatoes.com/m/the_king_of_kings is 404
https://www.rottentomatoes.com/m/journey_with_papa is 404
https://www.rottentomatoes.com/m/the_pursuit_of_happyness is 404
https://www.rottentomatoes.com/m/the_carpetbaggers is 404
https://www.rottentomatoes.com/m/the_night_of_the_grizzly is 404
https://www.rottentomatoes.com/m/the_whole_wide_world is 404
https://www.rottentomatoes.com/m/dealin_with_idiots is 404
https://www.rottentomatoes.com/m/the_exorcist_(film_series) is 404
https://www.rottentomatoes.com/m/up_in_the_world is 404
https://www.rottentomatoes.com/m/diana is 404
https://www.rottentomatoes.com/m/white_wilderness is 404
https://www.rottentomatoes.com/m/oedipus_the_king is 404
https://www.rottentomatoes.com/m/doing_time_for_patsy_cline is 404
https://www.rottentomatoes.com/m/24_hours_in_the_life_of_a_

https://www.rottentomatoes.com/m/calamity_jane is 404
https://www.rottentomatoes.com/m/the_wilby_conspiracy is 404
https://www.rottentomatoes.com/m/the_groundstar_conspiracy is 404
https://www.rottentomatoes.com/m/the_spanish_prisoner is 404
https://www.rottentomatoes.com/m/the_sting is 404
https://www.rottentomatoes.com/m/the_prodigy is 404
https://www.rottentomatoes.com/m/pokémon_the_movie_genesect_and_the_legend_awakened is 404
https://www.rottentomatoes.com/m/atomic_blonde is 404
https://www.rottentomatoes.com/m/american_made is 404
https://www.rottentomatoes.com/m/the_family_man is 404
https://www.rottentomatoes.com/m/running_with_the_devil is 404
https://www.rottentomatoes.com/m/the_assassination_of_jesse_james_by_the_coward_robert_ford is 404
https://www.rottentomatoes.com/m/man_up is 404
https://www.rottentomatoes.com/m/lambada_(1990_american_film) is 404
https://www.rottentomatoes.com/m/mad_love is 404
https://www.rottentomatoes.com/m/windfall_in_athens is 404
https://www.rott

https://www.rottentomatoes.com/m/the_last_time_i_saw_archie is 404
https://www.rottentomatoes.com/m/a_perfect_couple is 404
https://www.rottentomatoes.com/m/awakenings is 404
https://www.rottentomatoes.com/m/the_terror_of_tiny_town is 404
https://www.rottentomatoes.com/m/the_happening is 404
https://www.rottentomatoes.com/m/the_terrornauts is 404
https://www.rottentomatoes.com/m/the_tournament is 404
https://www.rottentomatoes.com/m/a_touch_of_the_sun is 404
https://www.rottentomatoes.com/m/henry_v is 404
https://www.rottentomatoes.com/m/lolita is 404
https://www.rottentomatoes.com/m/house_of_bamboo is 404
Progress: 0.8999999999999999
https://www.rottentomatoes.com/m/little_murder is 404
https://www.rottentomatoes.com/m/deuce_bigalow_european_gigolo is 404
https://www.rottentomatoes.com/m/the_importance_of_being_earnest is 404
https://www.rottentomatoes.com/m/the_boston_strangler is 404
https://www.rottentomatoes.com/m/the_discreet_charm_of_the_bourgeoisie is 404
https://www.rottentoma

In [50]:
df = pd.DataFrame(data=compiledData, columns=['name','audience','critics','date','genre','director'])
df.to_pickle('all_data.pkl')

In [55]:
print(len(crazy))
print(progressCount)
print(failCount)
retrydf = pd.DataFrame(data=retryList)
retrydf.to_pickle('all_retry.pkl')

268
5771
1272


In [56]:
unpickled_data = pd.read_pickle('all_data.pkl')
print(unpickled_data)

                                   name audience critics          date  \
0              children_of_a_lesser_god       79      81   Oct 3, 1986   
1                            changeland       66      63   Jun 7, 2019   
2                    japanese_war_bride     None    None  Jan 29, 1952   
3                                8_mile       54      75   Nov 8, 2002   
4                                 ted_2       50      45  Jun 26, 2015   
...                                 ...      ...     ...           ...   
4494                  two_english_girls       82      86  Oct 15, 1972   
4495                      3_generations       44      32   May 5, 2017   
4496                    prometheus_2012       68      73   Jun 8, 2012   
4497                      mrs_doubtfire       77      72  Nov 24, 1993   
4498  national_treasure_book_of_secrets       67      36  Dec 21, 2007   

                                                  genre             director  
0       [Drama, Musical &amp; Pe

In [58]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)


In [60]:
client.list_database_names()

['admin', 'config', 'local', 'rottentomatoes']

In [62]:
db = client.rottentomatoes
db.list_collection_names()

['reviews']

In [77]:
reviews = db.reviews
df_but_dict = df.to_dict("records'")
#print(df_but_dict)
reviews.insert_many(df_but_dict)


[{'name': 'children_of_a_lesser_god', 'audience': '79', 'critics': '81', 'date': 'Oct 3, 1986', 'genre': ['Drama', 'Musical &amp; Performing Arts', 'Romance'], 'director': ['Randa Haines']}, {'name': 'changeland', 'audience': '66', 'critics': '63', 'date': 'Jun 7, 2019', 'genre': ['Comedy', 'Drama'], 'director': ['Seth Green']}, {'name': 'japanese_war_bride', 'audience': None, 'critics': None, 'date': 'Jan 29, 1952', 'genre': ['Classics', 'Drama'], 'director': ['King Vidor']}, {'name': '8_mile', 'audience': '54', 'critics': '75', 'date': 'Nov 8, 2002', 'genre': ['Drama'], 'director': ['Curtis Hanson']}, {'name': 'ted_2', 'audience': '50', 'critics': '45', 'date': 'Jun 26, 2015', 'genre': ['Comedy'], 'director': ['Seth MacFarlane']}, {'name': 'thirst_1949', 'audience': '21', 'critics': None, 'date': None, 'genre': ['Drama', 'Mystery &amp; Suspense'], 'director': ['Jeffery Scott Lando']}, {'name': 'good_boys', 'audience': None, 'critics': None, 'date': 'Jun 3, 2005', 'genre': ['Art House

In [104]:
print(reviews.find()[1])
#review.update_many({'audience': {$exists: True}},)
for x in reviews.find({'audience': {"$ne": None}}):
    reviews.update_one({'_id' : x['_id']}, {"$set": {'audience' : int(x['audience'])}})

{'_id': ObjectId('5e399111114451dbb2fc36d9'), 'name': 'changeland', 'audience': 66, 'critics': '63', 'date': 'Jun 7, 2019', 'genre': ['Comedy', 'Drama'], 'director': ['Seth Green']}


In [105]:
for x in reviews.find({'critics': {"$ne": None}}):
    reviews.update_one({'_id' : x['_id']}, {"$set": {'critics' : int(x['critics'])}})

In [114]:
for x in reviews.find({"$and":[ {'audience': {"$ne": None}},{'critics': {"$ne": None}}]}):
    if  x['audience'] - x['critics']  > 30:
        print(x['name'])

the_smurfs_2
mac_and_me
reform_school_girls
jacobs_ladder_2019
the_longest_yard_1974
midway_2019
the_lion_of_judah
alvin_and_the_chipmunks_chipwrecked
hocus_pocus_1993
the_upside
mixed_nuts
the_wedding_ringer
king_arthur_legend_of_the_sword
the_miracle_season
madeas_big_happy_family
gods_not_dead_2
believe_2016
star_wars_the_rise_of_skywalker
airborne_1993
two_moon_junction
crackers_1984
son_of_god
grumpier_old_men
resident_evil
collateral_beauty
barefoot_2014
vampire_academy
national_security_2003
one_small_hitch
step_up
mahogany
the_longest_ride
hitman_agent_47
miracles_from_heaven
the_flowers_of_war
the_resurrection_of_gavin_stone
super_troopers_2
think_like_a_man_too
meet_the_blacks
dark_phoenix
the_case_for_christ
the_single_moms_club
dracula_untold
the_river_thief
the_vanishing_of_sidney_hall
axl
men_in_black_international
the_mortal_instruments_city_of_bones
grown_ups_2
colombiana
ludwig
blood_ransom
transformers_age_of_extinction
lady_in_a_cage
the_vips
batman_v_superman_dawn_o